In [2]:
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

# First letter of last name, iterate through all later

letter = 'a'
# Setting this as a url so we can modularize later

url = "https://www.basketball-reference.com/players/{}/".format(letter)
# this is the HTML from the URL
soup = BeautifulSoup(urlopen(url))

In [3]:
url = "https://www.basketball-reference.com/players/a/adamsst01/gamelog/2020/"
soup = BeautifulSoup(urlopen(url))

headers = []
for header in soup.findAll('th')[0:30]:
    headers.append(header.get_text()) 
headers

['Rk',
 'G',
 'Date',
 'Age',
 'Tm',
 '\xa0',
 'Opp',
 '\xa0',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'GmSc',
 '+/-']

In [11]:
import time
import requests
import re
from IPython.display import clear_output


comments = re.compile("<!--|-->")
yearly_seasons = []
yearly_playoffs = []

header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0"}

#1979-2020?
for i in range(1980,2021):
    
    season_players_list = []
    playoff_players_list = []
    
    #Finds the list of players for each season
    url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html/".format(i)
    soup = BeautifulSoup(requests.get(url).text)
    rows = soup.findAll('tr',{'class':'full_table'})
    
    
    num_loaded = 0
    
    #goes through every player
    for player in rows:
        #loading bar stuff
        num_loaded = num_loaded +1
        done = round((50/len(rows))*num_loaded)
        left = 50 - done
        clear_output(wait=True)
        print("\r","[",'#'*done,' '*left,']',' ',num_loaded,'/',len(rows), ' ',i,sep = '',end = '', flush=True)
        
        #finds the name of each player, and a link to their season preformance page
        player_name = player.findAll('a')[0].get_text()
        player_link = "https://www.basketball-reference.com{}/gamelog/{}".format(player.findAll('a')[0]['href'][:-5],i)
        
        try:
            r = requests.get(player_link,headers = header)#gets html of season stats
        except:
            #incase connection is dropped
            time.sleep(6)
            r = requests.get(player_link, headers = header)#gets html of season stats
            
        r = comments.sub('',r.text) #theres a section where the data is in comments, so we remove the commenting
        
        df_list = pd.read_html(r) # this parses all the tables in webpages to a list
        
        player_season = df_list[-2] #player preformance during regular season
        player_playoffs = df_list[-1] #player preformance during playoffs
        
        player_season = player_season[(player_season.index+1) %21 !=0].reset_index(drop=True) #gets rid of headers interspersed in the table
        player_playoffs = player_playoffs[(player_playoffs.index+1) %21 !=0].reset_index(drop=True) #gets rid of headers interspersed in the table
        
        player_season.insert(0,'player_name',player_name) #inserts player name into data
        player_playoffs.insert(0,'player_name',player_name) # ^^
        
        player_season.insert(1,'season', i)
        player_playoffs.insert(1,'season',i)

        #Adds to our dataframe
        season_players_list.append(player_season)
        playoff_players_list.append(player_playoffs)
    
    season_data = pd.concat(season_players_list)
    playoff_data = pd.concat(playoff_players_list)
    
    yearly_seasons.append(season_data)
    yearly_playoffs.append(playoff_data)
    
    
all_seasons = pd.concat(yearly_seasons, ignore_index=True)
all_playoffs = pd.concat(yearly_playoffs, ignore_index=True)
all_seasons = all_seasons.drop([0,1],axis =1)
    
all_playoffs.to_csv("bball_playoffs.csv",index=False)
all_seasons.to_csv("bball_seasons.csv",index=False)

[##################################################] 529/529 2020

In [14]:
all_playoffs

,player_name,season,Rk,G,Date,Age,Tm,Unnamed: 5,Opp,Unnamed: 7,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,Kareem Abdul-Jabbar,1980,1,1,1980-04-08,32-358,LAL,NaN,PHO,W (+9),...,9,12,2,2,5,5,2,30,29.7,NaN
1,Kareem Abdul-Jabbar,1980,2,2,1980-04-09,32-359,LAL,NaN,PHO,W (+3),...,4,7,4,1,1,4,2,32,24.2,NaN
2,Kareem Abdul-Jabbar,1980,3,3,1980-04-11,32-361,LAL,@,PHO,W (+3),...,6,9,2,3,3,5,4,37,32.3,NaN
3,Kareem Abdul-Jabbar,1980,4,4,1980-04-13,32-363,LAL,@,PHO,L (-26),...,8,11,0,0,4,3,2,25,16.8,NaN
4,Kareem Abdul-Jabbar,1980,5,5,1980-04-15,32-365,LAL,NaN,PHO,W (+25),...,NaN,16,3,0,1,3,3,35,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609957,Ivica Zubac,2020,9,9,2020-09-07,23-173,LAC,@,DEN,W (+6),...,3,3,0,0,0,3,6,8,2.8,10
609958,Ivica Zubac,2020,10,10,2020-09-09,23-175,LAC,@,DEN,W (+11),...,5,9,0,0,2,1,4,11,9,12
609959,Ivica Zubac,2020,11,11,2020-09-11,23-177,LAC,NaN,DEN,L (-6),...,6,9,1,1,0,2,1,5,3,5
609960,Ivica Zubac,2020,12,12,2020-09-13,23-179,LAC,@,DEN,L (-13),...,8,12,1,0,2,1,3,2,3.3,11


KeyError: '[0 1] not found in axis'